# Load Sqlite Dataset

In [1]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
import warnings
warnings.filterwarnings('ignore')


import nltk
import string
import sqlite3

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [2]:
con = sqlite3.connect('database.sqlite')
con

In [4]:
filtered_data = pd.read_sql_query("""select * from reviews where score !=3 limit 5000""", con)
filtered_data

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
4995,5423,B00622CYVS,A17ASMX6QMO6XY,E. Harvill,0,1,2,1277424000,Not so tasty...,"My baby didn't seem into these dinners, so I t..."
4996,5424,B00622CYVS,A32DHN8U74GCAR,"Granola Girl ""michele j.""",0,1,4,1240790400,Food Delivery,This is great! Organic baby food options - de...
4997,5425,B00622CYVS,A2YHXAZLCLDT8D,"Mark Smith ""Food lover""",0,1,5,1236988800,Dinner time is Earths Best TIme !!,My little guy loves to try new foods..so this ...
4998,5426,B00622CYVS,A2NYT3UXUTBY23,C&GHoll,1,3,2,1249603200,Wrong item shipped,We ordered the Earth's best 2nd dinner variety...


In [5]:
filtered_data.shape

(5000, 10)

In [6]:
filtered_data.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...


In [7]:
filtered_data['Score'].value_counts()

5    3420
4     767
1     504
2     309
Name: Score, dtype: int64

In [9]:
def partition(x):
    if x <3:
        return 0
    return 1
# Changing reviews with score less than 3 to be negative (0) and more than 3 to be positive(1)
actualScore = filtered_data['Score']
PositiveNegative = actualScore.map(partition)
filtered_data['Score'] = PositiveNegative
print("Number of data points in our dataset", filtered_data.shape)
filtered_data.head(10)

Number of data points in our dataset (5000, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,1,1342051200,Nice Taffy,I got a wild hair for taffy and ordered this f...
6,7,B006K2ZZ7K,A1SP2KVKFXXRU1,David C. Sullivan,0,0,1,1340150400,Great! Just as good as the expensive brands!,This saltwater taffy had great flavors and was...
7,8,B006K2ZZ7K,A3JRGQVEQN31IQ,Pamela G. Williams,0,0,1,1336003200,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...
8,9,B000E7L2R4,A1MZYO9TZK0BBI,R. James,1,1,1,1322006400,Yay Barley,Right now I'm mostly just sprouting this so my...
9,10,B00171APVA,A21BT40VZCCYT4,Carol A. Reed,0,0,1,1351209600,Healthy Dog Food,This is a very healthy dog food. Good for thei...


In [10]:
filtered_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [18]:
#Reference Dataset
display = pd.read_sql_query(""" 
select UserId, ProductId, ProfileName, Time, Score, Text, count(*) from reviews 
group by UserID
Having count(*)>1""", con)


In [21]:
display.shape

(80668, 7)

In [22]:
display.head

<bound method NDFrame.head of                    UserId   ProductId               ProfileName        Time  \
0      #oc-R115TNMSPFT9I7  B005ZBZLT4                   Breyton  1331510400   
1      #oc-R11D9D7SHXIJB9  B005HG9ESG    Louis E. Emory "hoppy"  1342396800   
2      #oc-R11DNU2NBKQ23Z  B005ZBZLT4          Kim Cieszykowski  1348531200   
3      #oc-R11O5J5ZVQE25C  B005HG9ESG             Penguin Chick  1346889600   
4      #oc-R12KPBODL2B5ZD  B007OSBEV0     Christopher P. Presta  1348617600   
...                   ...         ...                       ...         ...   
80663       AZZNK89PXD006  B001J3ZUQ6                     Etude  1269648000   
80664       AZZTH6DJ0KSIP  B00014IVPQ           Crystal Caccamo  1304208000   
80665       AZZU1VEO8KUXH  B003FDC2I2             Mia P "Mia P"  1317513600   
80666       AZZU4D6TZ2L6J  B0029XITW2  Sherry King "llamasmama"  1247875200   
80667       AZZV9PDNMCOZW  B001E5E2QI  Anne B. Ward "Dr Anne B"  1310428800   

       Score         

In [23]:
display['count(*)'].sum()

393063

In [24]:
#Actual Dataset
filtered_data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...


In [25]:
# Sorting the dataset into ascending order
sorted_data = filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False,
                                       kind='quicksort')

In [26]:
sorted_data.shape

(5000, 10)

In [27]:
# remove duplicate data in case exist
final = sorted_data.drop_duplicates(subset={'UserId','ProductId','Time','Text'},
                                   keep='first', inplace=False)

In [28]:
final.shape

(4994, 10)

In [29]:
final.head(100)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,1,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,1,961718400,Great Product,This was a really good idea and the final prod...
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,1,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...
...,...,...,...,...,...,...,...,...,...,...
932,1011,B0002MKFEM,A3QLX72AO0DD5Z,Carlito Picache,1,2,1,1226361600,Way too salty,I tried this and I found it too salty.<br />Pl...
4090,4433,B0002NYO98,A376TWN7I4HMZ8,helios,0,1,1,1324252800,Exaclty what i ordered,"Again, exactly what I ordered. No fuss, no mus..."
4089,4432,B0002NYO98,A5DVX3B075B09,Patricia Kays,0,0,1,1338940800,LOVELY JUNIPER BERRIES,"Dried berries, still with texture and the quin..."
4270,4642,B0002NYO9I,A376TWN7I4HMZ8,helios,0,1,1,1324252800,Exaclty what i ordered,"Again, exactly what I ordered. No fuss, no mus..."


In [30]:
final['Score'].value_counts()

1    4184
0     810
Name: Score, dtype: int64

In [31]:
final['HelpfulnessNumerator'].sum()

7854

In [32]:
final['HelpfulnessDenominator'].sum()

10068

In [33]:
final = final[final.HelpfulnessNumerator <= final.HelpfulnessDenominator]

In [34]:
final.shape

(4994, 10)

# Text Preprocessing

In [35]:
# Printing some sample review on the text column
sent_0 = final['Text'].values[0]
print(sent_0)
print("="*50)

sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

sent_1500 = final['Text'].values[1500]
print(sent_1500)
print("="*50)

sent_2757 = final['Text'].values[2757]
print(sent_2757)
print("="*50)

sent_4900 = final['Text'].values[4900]
print(sent_4900)
print("="*50)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
I like Kettle chips but was really disappointed with this order because they were over cooked.  I bought a bigger sized bag from Costco and the chips were all cooked perfectly. So I don't know why these were different.<br />Will never buy these chips from here again.
These cookies are both yummy and well priced. I recommend them. Other cookies I like are Mrs. Fields(packaged), Chips ahoy soft baked, fig newtons, oreos, and elf cookies.
It felt weird ordering 24 bags of chips off the internet that I had never even tried before, but I am SO GLAD I DID. They are delicious! I've tried 4 flavors so far & have loved each one. These are a win. :)
I like the slight pineapple flavor in this better than the plain coconut water. Good 

In [36]:
# filtering is required
import re
sent_0 = re.sub(r"http\S+","",sent_0)
sent_1000 = re.sub(r"http\S+","",sent_1000)
sent_1500 = re.sub(r"http\S+","",sent_1500)
sent_2757 = re.sub(r"http\S+","",sent_2757)
sent_4900 = re.sub(r"http\S+","",sent_4900)
print(sent_0)
print("*******"*10)
print(sent_1000)
print("*******"*10)
print(sent_1500)
print("*******"*10)
print(sent_2757)
print("*******"*10)
print(sent_4900)
print("*******"*10)

Why is this $[...] when the same product is available for $[...] here?<br /> /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.
**********************************************************************
I like Kettle chips but was really disappointed with this order because they were over cooked.  I bought a bigger sized bag from Costco and the chips were all cooked perfectly. So I don't know why these were different.<br />Will never buy these chips from here again.
**********************************************************************
These cookies are both yummy and well priced. I recommend them. Other cookies I like are Mrs. Fields(packaged), Chips ahoy soft baked, fig newtons, oreos, and elf cookies.
**********************************************************************
It felt weird ordering 24 bags of chips off the internet that I had never even tried before, but I am SO GLAD I DID. They are delicious! I've tried 

In [37]:
from bs4 import BeautifulSoup

In [38]:
soup = BeautifulSoup(sent_0, 'lxml')
text = soup.get_text()
print(text)
print("="*50)

Why is this $[...] when the same product is available for $[...] here? />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [39]:
text = re.sub('[^a-zA-Z0-9]+',' ', text)
text = text.lower()
print(text)

why is this when the same product is available for here the victor m380 and m502 traps are unreal of course total fly genocide pretty stinky but only right nearby 


In [40]:
def decontracted(phrase):
    # spesific - you may need SME support
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r" can\'t", "cannot", phrase)
    phrase = re.sub(r"don\'t", "do not", phrase)
    phrase = re.sub(r"n\'t", "not", phrase)
    phrase = re.sub(r"\'s'", "is", phrase)
    phrase = re.sub(r"\'d", "would", phrase)
    phrase = re.sub(r"\'ll", "will", phrase)
    phrase = re.sub(r"\'ve", "have", phrase)
    phrase = re.sub(r"\'t", "not", phrase)
    phrase = re.sub(r"\'m", "am", phrase)
    phrase = re.sub(r"\'re", "are", phrase)
    return phrase

In [41]:
sent_1000 = final['Text'].values[1000]
print(sent_1000)
print("="*50)

I like Kettle chips but was really disappointed with this order because they were over cooked.  I bought a bigger sized bag from Costco and the chips were all cooked perfectly. So I don't know why these were different.<br />Will never buy these chips from here again.


In [42]:
sent_1000 = decontracted(sent_1000)
print(sent_1000)
print("="*50)

I like Kettle chips but was really disappointed with this order because they were over cooked.  I bought a bigger sized bag from Costco and the chips were all cooked perfectly. So I do not know why these were different.<br />Will never buy these chips from here again.


In [43]:
sent_0 = final['Text'].values[0]
print(sent_0)

Why is this $[...] when the same product is available for $[...] here?<br />http://www.amazon.com/VICTOR-FLY-MAGNET-BAIT-REFILL/dp/B00004RBDY<br /><br />The Victor M380 and M502 traps are unreal, of course -- total fly genocide. Pretty stinky, but only right nearby.


In [44]:
# combine all the text 
sent_0 = re.sub('[^a-zA-Z0-9]+',' ', sent_0)
print(sent_0)

Why is this when the same product is available for here br http www amazon com VICTOR FLY MAGNET BAIT REFILL dp B00004RBDY br br The Victor M380 and M502 traps are unreal of course total fly genocide Pretty stinky but only right nearby 


In [45]:
final.head(1)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...


In [50]:
from nltk.corpus import stopwords

In [51]:
stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

In [52]:
# combining all the above text preprocessing
from tqdm import tqdm

preprocessed_review = []


for sentences in tqdm(final['Text'].values):
    sentences = re.sub(r"http\S+","",sentences)
    sentences = BeautifulSoup(sentences, 'lxml').get_text()
    sentences = decontracted(sentences)
    sentences = re.sub('[^a-zA-Z0-9]+',' ', sentences)
    #sentences = sentences.split()
    sentences = ' '.join(e.lower() for e in sentences.split() if e.lower() not in stopwords)
    preprocessed_review.append(sentences.strip())

100%|██████████| 4994/4994 [00:01<00:00, 2672.51it/s]


In [53]:
preprocessed_review[0]

'product available victor m380 m502 traps unreal course total fly genocide pretty stinky right nearby'

In [54]:
preprocessed_review[1500]

'cookies yummy well priced recommend cookies like mrs fields packaged chips ahoy soft baked fig newtons oreos elf cookies'

In [55]:
preprocessed_review[1000]

'like kettle chips really disappointed order cooked bought bigger sized bag costco chips cooked perfectly not know different never buy chips'

# Bag of words

In [61]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
count_vect.fit(preprocessed_review)
print("some features name", count_vect.get_feature_names_out()[1:20])
print("="*50)
final_counts = count_vect.transform(preprocessed_review)
print("The type of count vectorizer", type(final_counts))
print("The shape of the text BOW Vectorizer", final_counts.get_shape())
print("The number of unique words", final_counts.get_shape()[1])

some features name ['000' '000kwh' '002' '0100' '0174' '02' '03' '03510'
 '042608460503serving' '0472066978' '06' '0738551856' '09' '090' '0g'
 '0gcholesterol' '0gprotein' '0mg' '0mgsodium']
The type of count vectorizer <class 'scipy.sparse._csr.csr_matrix'>
The shape of the text BOW Vectorizer (4994, 13557)
The number of unique words 13557


# N-gram or Bag of N-gram

In [62]:
#from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(ngram_range=(1, 2),max_features=5000)
final_bigrams = count_vect.fit_transform(preprocessed_review)

print("The type of count vectorizer", type(final_bigrams))
print("The shape of the text BOW Vectorizer", final_bigrams.get_shape())
print("The number of unique words", final_bigrams.get_shape()[1])

The type of count vectorizer <class 'scipy.sparse._csr.csr_matrix'>
The shape of the text BOW Vectorizer (4994, 5000)
The number of unique words 5000


# TF-IDF

In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer()
tf_idf_vect.fit(preprocessed_review)
print("some features name", tf_idf_vect.get_feature_names_out()[1:20])
print("="*50)
final_counts_tfidf = tf_idf_vect.transform(preprocessed_review)
print("The type of count vectorizer", type(final_counts_tfidf))
print("The shape of the text BOW Vectorizer", final_counts_tfidf.get_shape())
print("The number of unique words", final_counts_tfidf.get_shape()[1])

some features name ['000' '000kwh' '002' '0100' '0174' '02' '03' '03510'
 '042608460503serving' '0472066978' '06' '0738551856' '09' '090' '0g'
 '0gcholesterol' '0gprotein' '0mg' '0mgsodium']
The type of count vectorizer <class 'scipy.sparse._csr.csr_matrix'>
The shape of the text BOW Vectorizer (4994, 13557)
The number of unique words 13557


In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
x = tfidf.fit_transform(preprocessed_review).toarray()

In [66]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
x.shape

(4994, 13557)

In [68]:
final.shape

(4994, 10)

In [69]:
final.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [70]:
y = final['Score']

In [71]:
y

2546    1
2547    1
1145    1
1146    1
2942    1
       ..
711     1
710     1
709     1
713     1
1362    0
Name: Score, Length: 4994, dtype: int64

In [72]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

In [75]:
from sklearn.naive_bayes import MultinomialNB
Food_review_model = MultinomialNB().fit(x_train, y_train)
y_pred_train_nb = Food_review_model.predict(x_train)
y_pred_test_nb = Food_review_model.predict(x_test)

In [76]:
print(accuracy_score(y_train, y_pred_train_nb))
print("******************"*10)
print(accuracy_score(y_test, y_pred_test_nb))

0.8365456821026283
************************************************************************************************************************************************************************************
0.8448448448448449


In [127]:
# Check your model performance

test1 = ['The pizza had a perfect balance of cheese, sauce, and toppings.']
test2 = ['The fish was not fresh; it had a slightly off smell and taste.']


In [126]:
array_2d_test1

array([['The pizza had a perfect balance of cheese, sauce, and toppings.']],
      dtype='<U63')

In [119]:
from sklearn.ensemble import RandomForestClassifier
Food_review_model = RandomForestClassifier().fit(x_train, y_train)
y_pred_train_rf = Food_review_model.predict(x_train)
y_pred_test_rf =  Food_review_model.predict(x_test)

In [120]:
print(accuracy_score(y_train, y_pred_train_rf))
print("******************"*10)
print(accuracy_score(y_test, y_pred_test_rf))

1.0
************************************************************************************************************************************************************************************
0.8608608608608609


In [121]:
from sklearn.linear_model import LogisticRegression
Food_review_model= LogisticRegression().fit(x_train, y_train)
y_pred_train_lr = Food_review_model.predict(x_train)
y_pred_test_lr = Food_review_model.predict(x_test)

In [122]:
print(accuracy_score(y_train, y_pred_train_lr))
print("******************"*10)
print(accuracy_score(y_test, y_pred_test_lr))

0.8848560700876095
************************************************************************************************************************************************************************************
0.8598598598598599


In [130]:
# pipeline package to merge nlp and machine learning together and get the output
from sklearn.pipeline import Pipeline 


In [131]:
classifier = Pipeline([('tfidf',TfidfVectorizer() ), 
                       ('classifier',RandomForestClassifier())])

In [132]:
classifier

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])